In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
import json



class OpinionDistribution(object):
    def __init__(self, 
                 trends: dict, 
                 p_o,
                 p_p,
                 iteration: int | str = -1, 
                 values: str = "probabilities"):
        
        """
        :param model: The model object
        :param trends: The computed simulation trends
        :param iteration: The iteration number or the string "last" for plotting final state
        :param values: The iteration number or the string "last" for plotting final state
        """
        
        self.trends = trends
        self.iteration = iteration
        
        if iteration == 'last':
            self.it = self.trends[-1]['iteration']
            self.ops = self.trends[-1]['status']
        else:
            self.ops = self.trends[iteration]['status'].values()
        
        if values == 'probabilities':
            weights = np.array([el for el in self.ops])
            self.values = p_o * weights + p_p * (1-weights)
            
        elif values == 'weights':
            self.values = np.array([el for el in self.ops])
    
    def get_values(self):
        return self.values
        
    def plot(self, filename=None, ax = None, values: str = "probabilities"):   
           if ax is None:
               fig, ax = plt.subplots(figsize=(10, 6))    
           ax = sns.histplot(self.get_values(), bins = 50, color='lightblue', alpha=1.0, stat='percent')
           ax.set_xlabel(r'$p_{i,T}$')
           ax.set_ylabel('% agents')
           ax.set_title(f'Final {values} distribution of optimist model')
           ax.set_xlim(0.0, 1.0)
           plt.tight_layout()
           if filename is not None:
               plt.savefig(filename, dpi=300, facecolor='white', bbox_inches='tight')
           else:
               plt.show()
               
           plt.close()

In [ ]:
import json
import os 
from tqdm.notebook import tqdm  # Use tqdm for Jupyter Notebook


values = ['weights', 'probabilities']
NLs = [0]

for nl in NLs:
    print(f'doing {nl} lobbyists')
    
    path = f'../results/balanced_budgets/{nl}_lobbyists/'
    filename = os.path.join(path, 'config.json')
    
    with open(filename, 'r') as f: #qua va messo il path del file initial_config.json
        params = json.load(f)
    
    total_iterations = len(values) * len(params['lambda_values']) * len(params['phi_values']) * params['nruns']
    with tqdm(total=total_iterations, desc="Processing", unit="iteration") as pbar:

        for value in values:
            for _, (lambda_v, phi_v) in enumerate([(l, p) for l in params['lambda_values'] for p in params['phi_values']]):    
                paramspath = os.path.join(path, f'{lambda_v}_{phi_v}/')        
                for run in range(params['nruns']):
                    runpath = os.path.join(paramspath, str(run))
                    if not os.path.exists(runpath+f'/{value}_final_distribution.png'):
                        with open(runpath+'/status.json', 'r') as f:
                            trends = json.load(f)
                        
                        od = OpinionDistribution(trends, params['p_o'], params['p_p'], values=value)
                        od.plot(runpath+f'/{value}_final_distribution.png',values=value)
                    
                    pbar.update(1)
        
    print(f'finished {nl} lobbyists')

doing 0 lobbyists


Processing:   0%|          | 0/4 [00:00<?, ?iteration/s]

finished 0 lobbyists
